In [35]:
!pip freeze | grep scikit-learn

In [36]:
import sklearn

In [38]:
sklearn.__version__

In [1]:
import pickle
import pandas as pd

In [2]:
year = 2022
month = 2

input_file = f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet'
output_file = f'output/yellow_tripdata_{year:04d}-{month:02d}.parquet'

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [4]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [5]:
#df = read_data('https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2022-02.parquet')
#df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet')

In [6]:
df = read_data(input_file)
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [8]:
df.head()

In [9]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [10]:
y_pred

In [11]:
import seaborn as sns
import matplotlib.pyplot as plt

In [12]:
sns.histplot(y_pred, label='prediction')
sns.histplot(df.duration, label='actual')

plt.legend();

### Q1. Standard deviation

In [13]:
y_pred.mean()

In [14]:
y_pred.std()

### Q2. Preparing the output

In [15]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predicted_duration'] = y_pred

In [18]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [33]:
%ls output